In [1]:
import requests as rq
import lxml.html as lx
import pandas as pd
import re

In [2]:
#read html + make df for general stats


url = 'https://www.espn.com/nba/team/stats/_/name/sac/salary'
names = pd.read_html(url)[0]
stats = pd.read_html(url)[1]
kings_stats = pd.concat([names,stats],axis = "columns")

In [3]:
# read html + make df for shooting stats


shoot = pd.read_html(url)[3]
kings_shoot_stats = pd.concat([names,shoot], axis = "columns")

In [4]:
# clean up "name" and drop total from df


words = []
for i in kings_shoot_stats["Name"]:
    words.append(i.split()[0:2])
words = pd.DataFrame(words)
words.columns = ["First","Last"]
words["Names"] = words['First']+" "+ words["Last"]
words = words.drop("First", axis = 1).drop("Last", axis = 1)
kings_shoot_stats["Name"] = words
kings_shoot_stats = kings_shoot_stats.drop(16)

In [5]:
#create salary df
url2 = "https://www.espn.com/nba/team/roster/_/name/sac/salary"
kings_salary_raw = pd.read_html(url2)[0]

In [6]:
#Strips player number from each players name

for i in range(0,len(kings_salary_raw)):
    kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2764\3064642318.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kings_salary_raw["Name"][i] = re.sub(r"\d+", "", str(kings_salary_raw["Name"][i]))


In [7]:
#Cleans dataframe to only player name + salary
kings_salary = kings_salary_raw.set_index("Name").drop('Unnamed: 0', axis = 1)
kings_salary.columns
kings_salary = kings_salary.drop(['POS', 'Age', 'HT', 'WT', 'College'], axis = 1)

In [22]:
#combine dfs. final kings 2022 df


kings_stats_22 = kings_shoot_stats.join(kings_salary, on = "Name")

In [27]:
#create dict for 2018-2023 team stats


from time import sleep



tab = pd.DataFrame()
#Empty dictionary
tab_dict = {}
#Fetches data from 2018-2019 to 2022-2023 seasons
for i in range(2019, 2024):
    #Changes the url for each season
    url_loop = "https://www.espn.com/nba/stats/team/_/season/"+str(i)+"/seasontype/2"
    sleep(.02)
    #Gets dataframe with only team names
    team_names = pd.read_html(url_loop)[0]
    #Gets dataframe with all corresponding team stats
    team_stats = pd.read_html(url_loop)[1]
    #Concatenates the two above data frames
    tab = pd.concat([team_names,team_stats],axis = "columns")
    #Labels dataframe by year
    tab = tab.style.set_caption(str(i-1)+"-"+str(i))
    name = "tab_"+str(i-1)
    #Adds to dictionary with 'tab_20xx' as key and the corresponding dataframe as the value
    tab_dict[name] = tab


TypeError: 'Styler' object is not subscriptable

In [31]:
type(tab_dict['tab_2021'])

pandas.io.formats.style.Styler

In [10]:
#create dict for 2018-2023 opponent team stats


from time import sleep



tab = pd.DataFrame()
#Empty dictionary
tab_dict_op = {}
#Fetches data from 2018-2019 to 2022-2023 seasons
for i in range(2019, 2024):
    #Changes the url for each season
    url_loop_op = "https://www.espn.com/nba/stats/team/_/view/opponent/season/"+str(i)+"/seasontype/2"
    sleep(.02)
    #Gets dataframe with only team names
    team_names = pd.read_html(url_loop_op)[0]
    #Gets dataframe with all corresponding team stats
    team_stats = pd.read_html(url_loop_op)[1]
    #Concatenates the two above data frames
    tab = pd.concat([team_names,team_stats],axis = "columns")
    #Labels dataframe by year
    tab = tab.style.set_caption(str(i-1)+"-"+str(i))
    name = "tab_op_"+str(i-1)
    #Adds to dictionary with 'tab_20xx' as key and the corresponding dataframe as the value
    tab_dict_op[name] = tab

tab_dict_op

{'tab_op_2018': <pandas.io.formats.style.Styler at 0x201766cc0a0>,
 'tab_op_2019': <pandas.io.formats.style.Styler at 0x201744f35b0>,
 'tab_op_2020': <pandas.io.formats.style.Styler at 0x201766cc490>,
 'tab_op_2021': <pandas.io.formats.style.Styler at 0x20174864be0>,
 'tab_op_2022': <pandas.io.formats.style.Styler at 0x201763927f0>}

In [19]:
url3 = "https://www.espn.com/nba/standings/_/season/2023"
names = pd.read_html(url3)[0]
stats = pd.read_html(url3)[1]
names.iloc[:,0][1].split()

,1UTAHUtah Jazz
0,2PHXPhoenix Suns
1,3MEMMemphis Grizzlies
2,4PORPortland Trail Blazers
3,5DALDallas Mavericks
4,6DENDenver Nuggets
5,7LACLA Clippers
6,8SACSacramento Kings
7,9NONew Orleans Pelicans
8,10MINMinnesota Timberwolves
9,GSGolden State Warriors


In [12]:
team1 = names.columns[0]
names.columns = ["Team"]
new_row = pd.DataFrame({names.columns[0]:team1}, index=[0])
names = pd.concat([new_row,names.loc[:]]).reset_index(drop=True)
for i in range(0, len(names.iloc[:,0])):
    if i < 8:
        names.iloc[i,0] = names.iloc[i,0][4:]
    elif i == 8:
        names.iloc[i,0] = names.iloc[i,0][3:]
    elif i == 9:
        names.iloc[i,0] = names.iloc[i,0][5:]
    else:
        names.iloc[i,0] = names.iloc[i,0][3:]
temp = names.join(stats.iloc[:,0:2])
temp

,Team,W,L
0,Boston Celtics,13,3
1,Milwaukee Bucks,11,4
2,Atlanta Hawks,10,6
3,Cleveland Cavaliers,9,6
4,Indiana Pacers,9,6
5,Washington Wizards,9,7
6,Toronto Raptors,9,8
7,Philadelphia 76ers,8,8
8,New York Knicks,8,9
9,Brooklyn Nets,7,9


In [20]:
url4 = "https://www.espn.com/nba/standings/_/season/2022"
names = pd.read_html(url4)[0]
stats = pd.read_html(url4)[1]
#names.iloc[:,0][11].split()
names

,1z --MIAMiami Heat
0,2y --BOSBoston Celtics
1,3y --MILMilwaukee Bucks
2,4x --PHIPhiladelphia 76ers
3,5x --TORToronto Raptors
4,6x --CHIChicago Bulls
5,7xp --BKNBrooklyn Nets
6,8xp --ATLAtlanta Hawks
7,9pb --CLECleveland Cavaliers
8,10pb --CHACharlotte Hornets
9,e --NYNew York Knicks


In [14]:
#Dataframes available:
kings_stats
kings_shoot_stats
kings_salary
tab_dict #tab_dict['tab_2019]...tab_dict['tab_2022']
tab_dict_op

{'tab_op_2018': <pandas.io.formats.style.Styler at 0x201766cc0a0>,
 'tab_op_2019': <pandas.io.formats.style.Styler at 0x201744f35b0>,
 'tab_op_2020': <pandas.io.formats.style.Styler at 0x201766cc490>,
 'tab_op_2021': <pandas.io.formats.style.Styler at 0x20174864be0>,
 'tab_op_2022': <pandas.io.formats.style.Styler at 0x201763927f0>}